In [ ]:
# get concept ids for each triple for the QA graphs in CSQA, OBQA, and MedQA
# output files are used in generate_graph_sps.bat to generate graph embeddings

import torch
import numpy as np
import pickle
import os


graph_path = '../data/'
for dataset in ('csqa', 'obqa', 'medqa'):
    for split in ('dev', 'test', 'train'):
        graph_file = graph_path+dataset+'/graph/'+split+'.graph.adj.pk'
        with open(graph_file, 'rb') as f:
            adj_concept_pairs = pickle.load(f)

        prune = True
        max_node_num = 200
        rel = []
        subjects = []
        objects = []
        all_concepts = []
        for idx, _data in enumerate(adj_concept_pairs):
                if dataset == 'medqa':
                    adj, concepts, qm, am = _data
                else:
                    adj, concepts, qm, am, cid2score = _data['adj'], _data['concepts'], _data['qmask'], _data['amask'], _data['cid2score']
                    
                num_concept = min(len(concepts), max_node_num)

                # prune nodes to 200 concepts (they are already sorted from highest relevance to lowest)
                concepts = concepts[:num_concept]

                ij = torch.tensor(adj.row, dtype=torch.int64) #(num_matrix_entries, ), where each entry is coordinate
                k = torch.tensor(adj.col, dtype=torch.int64)  #(num_matrix_entries, ), where each entry is coordinate
                n_node = adj.shape[1]
                if n_node > 0:
                    half_n_rel = adj.shape[0] // n_node
                    i, j = ij // n_node, ij % n_node

                    if prune:
                        # prune edges to the top 200 concepts
                        mask = (j < num_concept) & (k < num_concept)
                        i, j, k = i[mask], j[mask], k[mask]
                else:
                    print(idx)
                
                rel.append(i.numpy())
                subjects.append(j.numpy())
                objects.append(k.numpy())
                all_concepts.append(concepts)
                
        rel = np.array(rel, dtype=object)
        subjects = np.array(subjects, dtype=object)
        objects = np.array(objects, dtype=object)
        all_concepts = np.array(all_concepts, dtype=object)

        output_dir = graph_path+dataset+'/graph/'

        np.savez(output_dir+split+'_graph_ids_pruned', rels=rel, subjects=subjects, objects=objects, concept_ids = all_concepts)

C:\Users\ryanl\AppData\Local\Temp\ipykernel_32804\624312358.py:12: DeprecationWarning: Please use `coo_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.coo` namespace is deprecated.
  adj_concept_pairs = pickle.load(f)


Option 3
qm [ True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
am [False False False False  True  True  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False Fal

In [1]:
# testing number of concepts in each graph

import pickle
import numpy as np
dataset = 'medqa'
split = 'train'
with open('C:/Users/ryanl/OneDrive - University of Waterloo/Documents/University/Masters/CNRG/qaspa/data/'+dataset+'/graph/'+split+'.graph.adj.pk', 'rb') as f:
    adj_concept_pairs = pickle.load(f)

concepts_len = []
for idx, _data in enumerate(adj_concept_pairs):
        if dataset == 'medqa':
             adj, concepts, qm, am = _data
        else:
            adj, concepts, qm, am, cid2score = _data['adj'], _data['concepts'], _data['qmask'], _data['amask'], _data['cid2score']
        
        concepts_len.append(concepts.shape)
concepts_len = np.array(concepts_len)

concepts_len[concepts_len > 200].shape[0] / concepts_len.shape[0]

# import numpy as np
# graphs = np.load('csqa_dev_graph_ids.npz', allow_pickle=True)

# for graph in graphs['concept_ids']:
#     print(graph.shape)

C:\Users\ryanl\AppData\Local\Temp\ipykernel_10884\1750699269.py:8: DeprecationWarning: Please use `coo_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.coo` namespace is deprecated.
  adj_concept_pairs = pickle.load(f)


0.5270681862841423

In [ ]:
# split='train'
# dataset='csqa'
# graph = np.load('../'+dataset+'/'+split+'_graph_ids_pruned.npz', allow_pickle=True)
# total_sub = 0
# total_obj = 0
# for sub, obj in zip(graph['subjects'], graph['objects']):
#     total_sub += np.sum(sub > 200)
#     total_obj += np.sum(obj > 200)

# print(total_sub)
# print(total_obj)

0
0
